In [3]:
from forgeflow.synth import SynthConfig, make_production
from forgeflow.features import add_date_parts, one_hot_encode

prod_df = make_production(SynthConfig(n_days=30))
feat_df = add_date_parts(prod_df, "date")
feat_df = one_hot_encode(feat_df, ["plant", "product"])

feat_df.head()
feat_df.shape
feat_df[['plant_campinas', 'plant_manaus', 'plant_recife']].sum()
feat_df[['product_coffee_maker', 'product_microwave', 'product_vacuum']].sum()




product_coffee_maker    90
product_microwave       90
product_vacuum          90
dtype: int64